In [65]:
import csv
import os.path 
import json 
import numpy as np
import datetime
import operator

pathPrefix = ''
prefix = pathPrefix + 'chart_cluster_data_'
ending = '.csv'

headerKeys = ['pub_year', 'appl_year', 'grant_year', 'years_to_publish', 
              'years_to_grant', 'patent_doc_kind', 'appl_change_count','number_of_claims']

headerValue = 'count'
 
def loadStatsFromOneFile(clusterStr): 
    fName = ''
    
    if(len(clusterStr)==2) and ('x' not in clusterStr):
        fName = prefix + clusterStr[0] +"_"+ clusterStr[1] + ending   
    #print fName
    
    dicts=[]
    for keystr in headerKeys:
        dicts.append({}) 
    
    if(fName == ''):
        return dicts
    
    with open(fName) as csvfile:
        spamreader = csv.reader(csvfile, delimiter= ' ', quotechar='|')
        rowCount = -1 
        for row in spamreader:
            rowCount += 1
            jsonStr = str(row).replace("['",'').replace("']",'')
            jsons = json.loads(jsonStr)
            for j in jsons:  
                keystr = headerKeys[rowCount]
                if j[keystr] in dicts[rowCount]:
                    dicts[rowCount][j[keystr]] += int(j[headerValue])
                else:
                    dicts[rowCount][j[keystr]] = int(j[headerValue])

    return dicts

def mergeDicts(dict_full, dict_temp):
    for k,v in dict_temp.items():
        if len(str(k))<1:
            continue 
        if k in dict_full:
            dict_full[k] += v
        else:
            dict_full[k] = v
            
def loadStatsFromFiles(clusterStr): 
    clusters = ['0','1','2','3','4','5','6','7','8','9']
    files = []
    dicts=[]
    for keystr in headerKeys:
        dicts.append({}) 
    
    if clusterStr[0] == 'x':
        for x in clusters: 
            if(clusterStr[1] == 'x'):
                for xx in clusters:
                    files.append(x+xx)
            else:
                files.append(x+clusterStr[1])
    else:
        if(clusterStr[1] == 'x'):
            for xx in clusters:
                files.append(clusterStr[0]+xx)
        else:
            files.append(clusterStr[0]+clusterStr[1])

    for f in files: 
        temp = loadStatsFromOneFile(f)
        count = 0
        for d in dicts:
            mergeDicts(d, temp[count])
            count += 1

    return dicts


def saveToCsv(clusterStr, data_dict_original, header):
    fileToSave = prefix + clusterStr + '_' + header[0] + ending
    data_dict = sorted(data_dict_original.items(), key=operator.itemgetter(0))
    
    if os.path.isfile(fileToSave) == False:
        with open(fileToSave, 'w') as csvfile:
            fieldnames = header
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for key, value in data_dict: 
                writer.writerow({header[0]: key, header[1]: value})
    return fileToSave

def createStatsSummary(clusterStr):
    dicts = loadStatsFromFiles(clusterStr)
    files = []
    count = 0
    for d in dicts:
        f = saveToCsv(clusterStr, d, [headerKeys[count],headerValue])
        files.append(f)
        count += 1
        
    return files

In [66]:
createStatsSummary("1x")

['chart_cluster_data_1x_pub_year.csv',
 'chart_cluster_data_1x_appl_year.csv',
 'chart_cluster_data_1x_grant_year.csv',
 'chart_cluster_data_1x_years_to_publish.csv',
 'chart_cluster_data_1x_years_to_grant.csv',
 'chart_cluster_data_1x_patent_doc_kind.csv',
 'chart_cluster_data_1x_appl_change_count.csv',
 'chart_cluster_data_1x_number_of_claims.csv']

In [59]:
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
from plotly.graph_objs import Scatter, Layout, Bar
    
def plotDict(data_dict, yearlyData, fileName, offline):
    py.sign_in('yanglinfang', 'F5jaY29GDfD8TIenMz9p')
    date = plt.figure()
    x = []
    y = []
    for k in data_dict:
        if(yearlyData == True):
            x.append(datetime.datetime(k, 1, 1))
        else:
            x.append(k)
        y.append(data_dict[k])
        
    layout = Layout(
        title=fileName,
        xaxis=dict(
            autotick=True,
            ticks='outside',
            tickcolor='#000'
        ),
        yaxis=dict(
            autotick=True,
            ticks='outside',
            tickcolor='#000'
        )
    )
    
    if(yearlyData == True):
        layout.xaxis=dict(range=[datetime.datetime(2000, 1, 1), datetime.datetime(2018, 1, 1)]) 
    
    if offline == True: 
        plotly.offline.init_notebook_mode(connected=True)
        plotly.offline.iplot({
            "data": [Bar(x=x, y=y)],
            "layout": layout
        })
    else:
        print ("only supporting offline for now")

In [60]:
dicts = loadStatsFromFiles("1x")
count = 0
for d in dicts:
    keystr = headerKeys[count]
    if keystr in ['pub_year', 'appl_year', 'grant_year']:
        plotDict(d, True, keystr, True) 
    if keystr in ['years_to_publish', 'years_to_grant', 'patent_doc_kind', 'appl_change_count', 'number_of_claims']:
        plotDict(d, False, keystr, True) 
    count += 1
    

In [164]:
py.sign_in('yanglinfang', 'F5jaY29GDfD8TIenMz9p')

def results(symbol, trend1, trend2):
    data = web.DataReader(symbol, data_source='yahoo')
    data['Trend 1'] = data['Adj Close'].rolling(int(trend1)).mean()
    data['Trend 2'] = data['Adj Close'].rolling(int(trend2)).mean()
    #print data
    url = data[['Adj Close', 'Trend 1', 'Trend 2']].iplot(asUrl=True)
    print url 
    table = data.tail().to_html()

results('AAPL', 20, 100)

<plotly.tools.PlotlyDisplay object>
